# S03 T04: Feature Extraction and Classification

## The one with the libs

In [ ]:
from keras import models, utils
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score

In [ ]:
path_h5 = "/content/drive/My Drive/Piclkes Mel Spec/Melhor resultado/"
path = "/content/drive/My Drive/Arquivos 10 classes/Pickles Sem Augmentation/"

## The one who loads the training model and the test set

In [ ]:
# Lê o pickle e transforma em um numpy array
def read_pickle(name):
    with (open(name, 'rb')) as openfile:
        while True:
            try:
                one_instance = pickle.load(openfile)
            except EOFError:
                break
    one_instance = np.asanyarray(one_instance)
    return one_instance

In [ ]:
def pre_processing_data_to_keras_format(x, y):
    x = x.reshape(x.shape[0],x.shape[1],x.shape[2],1)
    y = utils.to_categorical(y)
    return x, y

In [ ]:
ls "/content/drive/My Drive/Arquivos 10 classes/Pickles Sem Augmentation/"

NEW_mel_x_Tests.pickle  NEW_mel_y_Tests.pickle  Teste_y_Teste_mel_int.pickle
NEW_mel_x_Train.pickle  NEW_mel_y_Train.pickle  Treino_y_Treino_mel_int.pickle


In [ ]:
#Para a extração
x_train = read_pickle(path+'NEW_mel_x_Train.pickle')
y_train = read_pickle(path+'Treino_y_Treino_mel_int.pickle')

# Para a extração e classificação
x_test = read_pickle(path+'NEW_mel_x_Tests.pickle')
y_test = read_pickle(path+'Teste_y_Teste_mel_int.pickle')

x_test, y_test = pre_processing_data_to_keras_format(x_test, y_test)
x_train, y_train = pre_processing_data_to_keras_format(x_train, y_train)

# Predição

In [ ]:
#Carrega o modelo
model = models.load_model(path_h5+'best_model_mel_ballroom.h5')

In [ ]:
# Realiza a predição
pred = model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
print('Acuracia da rede = ',accuracy_score(pred,np.argmax(y_test,axis=1)))
print('Kappa: ', cohen_kappa_score(pred,np.argmax(y_test,axis=1)))
print('Matriz de confusão da rede = \n', confusion_matrix(pred,np.argmax(y_test,axis=1)))

Acuracia da rede =  0.5559105431309904
Kappa:  0.5078161291394554
Matriz de confusão da rede = 
 [[367   1   1   9   0   0   0   3   0   4   0   0]
 [  0 434   1   3  19   4   0   4   0   2   6  13]
 [  3   0 439   1   0   1   0   8   0   0   0   2]
 [  8   0   1  29   0   0   0   1   0   0   0   1]
 [ 14   4   0   0 421   3   0  14   0   0   0   0]
 [ 15  15   1   0  17 393   0  10   0   4  28 102]
 [  6   1   4   0   2   6   0 380   0   1   0   0]
 [  8   7   6  11   1   4   0   4   0 406   2  23]
 [  0   1  10   0   0   5   0   5   0   0 175  26]
 [  0  10   2   0   3  16   0   0   0   1  26 198]
 [  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0]]


# The one with the feature extraction

In [ ]:
def extract_features(x_test, model):

    intermediate_layer_model = models.Model(inputs=model.input, outputs=model.get_layer(index = -2).output)
    features = intermediate_layer_model.predict(x_test)

    features = pd.DataFrame(data=features)
    
    return features

In [ ]:
path_save_pickle = "/content/drive/My Drive/Arquivos 10 classes/Deep Features/"
# path_save_csv = "/content/drive/My Drive/Piclkes Mel Spec/Arquivos CSV CNN/"

In [ ]:
#Utilizando a rede treinada para extrair as características dos espectrograms
x_train_features = extract_features(x_train,model)
x_test_features = extract_features(x_test,model)

#Salvando as características no formato pickle
pickle.dump(x_train_features, open(path_save_pickle+'x_train_deep_features_1.pickle', 'wb'))
pickle.dump(x_test_features, open(path_save_pickle+'x_test_deep_features_1.pickle', 'wb'))

# #Salvando as características no formato csv
# x_train_features.to_csv(path_save_csv+'x_train_deep_features.csv', index=False)
# x_test_features.to_csv(path_save_csv+'x_test_deep_labels.csv', index=False)